# Notebook (template) for Bollinger RSI strategy

## First we load the data

In [1]:
import pandas as pd

df = pd.read_json('AAPL.json')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_values(by='date', inplace=True)

df.head()

,id,symbol,open,high,low,close,volume,split_factor,adj_close,adj_high,adj_low,adj_open
date,,,,,,,,,,,,
2018-03-02 00:00:00+00:00,a92c9f2c-b7f8-4b2b-a392-2be6fbfb61db,AAPL,172.80,176.30,172.45,176.21,38453950,1,41.939318,41.960739,41.044409,41.127712
2018-03-05 00:00:00+00:00,850f750c-b0ac-449e-8817-b6ae57228003,AAPL,175.21,177.74,174.52,176.82,28401366,1,42.084503,42.303470,41.537085,41.701310
2018-03-06 00:00:00+00:00,261ee7f6-8849-4566-93a6-d10457c39820,AAPL,177.91,178.25,176.13,176.67,23788506,1,42.048801,42.424853,41.920277,42.343931
2018-03-07 00:00:00+00:00,cb447269-3b43-4308-b750-a5b4e7b1a8f2,AAPL,174.94,175.85,174.27,175.03,31703462,1,41.658469,41.853635,41.477583,41.637048
2018-03-08 00:00:00+00:00,42a4c183-4d0b-4f73-9b65-34ef288e54a0,AAPL,175.48,177.12,175.07,176.94,23163767,1,42.113063,42.155905,41.667989,41.765572


## Then we prepare the data for the `backtesting` library
We will work with the adjusted prices for backtesting.

In [2]:
df['Close'] = df['adj_close']
df['Open'] = df['adj_open']
df['High'] = df['adj_high']
df['Low'] = df['adj_low']
df['Volume'] = df['volume']

## Create the strategy leveraging the `backtesting` library
We will use the `BollingerBands` and `RSI` indicators from the `ta` library.

In [26]:
from backtesting import Backtest, Strategy
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

def ta_bollinger_lowerband(values, window, window_dev):
    close = pd.Series(values)
    return BollingerBands(close=close, window=window, window_dev=window_dev).bollinger_lband()

def ta_bollinger_upperband(values, window, window_dev):
    close = pd.Series(values)
    return BollingerBands(close=close, window=window, window_dev=window_dev).bollinger_hband()

def ta_momentum_rsi(values, window):
    close = pd.Series(values)
    return RSIIndicator(close=close, window=window).rsi()


class BollingerRsi(Strategy):
    
    bb_window = 25
    bb_window_dev = 2.5
    rsi_window = 2
    rsi_sell = 60
    rsi_buy = 45

    def init(self):
        self.bt_volatility_bbh = self.I(ta_bollinger_upperband, self.data.Close, self.bb_window, self.bb_window_dev)
        self.bt_volatility_bbl = self.I(ta_bollinger_lowerband, self.data.Close, self.bb_window, self.bb_window_dev)
        self.bt_momentum_rsi = self.I(ta_momentum_rsi, self.data.Close, self.rsi_window)

    def next(self):
        if (self.data.close > self.bt_volatility_bbh) and (self.bt_momentum_rsi > self.rsi_sell):
            self.position.close()
            self.sell()

        elif (self.data.close < self.bt_volatility_bbl) and (self.bt_momentum_rsi < self.rsi_buy):
            self.position.close()
            self.buy()


## Instantiate the strategy

In [27]:
bt = Backtest(df, BollingerRsi, cash=10_000, commission=.02, exclusive_orders=True)

## Run the backtest

In [28]:
stats=bt.run()

bt.plot()

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p7998', ...)

In [29]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-248,27,28,40.172207,41.272897,-272.971099,-0.027399,2018-04-11 00:00:00+00:00,2018-04-12 00:00:00+00:00,1 days
1,-240,28,29,40.447439,41.598967,-276.366800,-0.028470,2018-04-12 00:00:00+00:00,2018-04-13 00:00:00+00:00,1 days
2,-231,29,30,40.766988,41.658469,-205.932166,-0.021868,2018-04-13 00:00:00+00:00,2018-04-16 00:00:00+00:00,3 days
3,-226,30,31,40.825300,42.005960,-266.829265,-0.028920,2018-04-16 00:00:00+00:00,2018-04-17 00:00:00+00:00,1 days
4,-218,31,32,41.165841,42.320130,-251.635051,-0.028040,2018-04-17 00:00:00+00:00,2018-04-18 00:00:00+00:00,1 days
...,...,...,...,...,...,...,...,...,...,...
231,-1,512,519,58.050207,61.389347,-3.339140,-0.057522,2020-03-16 00:00:00+00:00,2020-03-25 00:00:00+00:00,9 days
232,-1,519,520,60.161560,60.353746,-0.192186,-0.003194,2020-03-25 00:00:00+00:00,2020-03-26 00:00:00+00:00,1 days
233,-1,520,521,59.146671,61.878993,-2.732322,-0.046196,2020-03-26 00:00:00+00:00,2020-03-27 00:00:00+00:00,1 days
234,-1,521,523,60.641413,62.576738,-1.935325,-0.031914,2020-03-27 00:00:00+00:00,2020-03-31 00:00:00+00:00,4 days


## Run an optimizer to see if we can improve that sucker
The target is SQN (System Quality Number) which is a measure of the strategy's performance.

In [30]:
stats, heatmap = bt.optimize(bb_window=[15, 20, 25, 30, 35], 
                             bb_window_dev=[1.0, 1.5, 2.0, 2.5], 
                             rsi_window=[10, 12, 14, 16, 18], 
                             rsi_sell=[55, 60, 65, 70, 75, 80], 
                             rsi_buy=[45, 40, 35, 30], 
                             return_heatmap=True)

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 2400 configurations.
  output = _optimize_grid()
/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [31]:
bt.plot()

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p9279', ...)

In [ ]:
stats

In [16]:
heatmap

window  window_dev  rsi_sell  rsi_buy
15      1.0         55        45        -8.844650
                              40        -8.844650
                              35        -8.844650
                              30        -8.844650
                    60        45        -9.000813
                                           ...   
35      2.5         75        30        -1.097454
                    80        45              NaN
                              40              NaN
                              35              NaN
                              30              NaN
Name: SQN, Length: 480, dtype: float64

In [17]:
heatmap.sort_values().dropna().iloc[-3:]

window  window_dev  rsi_sell  rsi_buy
30      2.0         80        40        -0.469143
35      2.0         80        40        -0.348017
30      2.0         80        45        -0.149618
Name: SQN, dtype: float64

In [ ]:
stats['_trades']

In [ ]:
stats['_strategy']